In [1]:
# Resolve paths from root project directory

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd, numpy
import pickle

In [4]:
DATE = "16-dec"
DATA_DIR = "../data/{}/".format(DATE)
EXPORT_DIR = "../data/dataframes/{}/".format(DATE)

In [5]:
from data_tools.dataframes import create_tweet_df, create_user_df, create_retweet_df, aggregate_counts_by_hour, aggregate_most_common_hashtags, create_media_df
from data_tools import load_crawled_terms 

## Load from cache

In [6]:
# retweet_df = pd.read_pickle(EXPORT_DIR + 'df_retweets.pickle')
# df_users = pd.read_pickle(EXPORT_DIR + 'df_users.pickle')
recent_tweet_df = pd.read_pickle(EXPORT_DIR + 'df_recent_tweets.pickle')

## Build from scratch

In [7]:
CRAWLED_TERMS = load_crawled_terms("../keywords-3nov.txt")

In [6]:
retweet_df = create_retweet_df(data_dir=DATA_DIR)
retweet_df.info()

Loading 25566698 json lines
(0%): 100000 lines in ../data/16-dec/parsed_retweets.json processed (0.34247398376464844 sec)
(1%): 200000 lines in ../data/16-dec/parsed_retweets.json processed (0.3321700096130371 sec)
(1%): 300000 lines in ../data/16-dec/parsed_retweets.json processed (0.37143874168395996 sec)
(2%): 400000 lines in ../data/16-dec/parsed_retweets.json processed (0.33399295806884766 sec)
(2%): 500000 lines in ../data/16-dec/parsed_retweets.json processed (0.40375328063964844 sec)
(2%): 600000 lines in ../data/16-dec/parsed_retweets.json processed (0.40134692192077637 sec)
(3%): 700000 lines in ../data/16-dec/parsed_retweets.json processed (0.39596986770629883 sec)
(3%): 800000 lines in ../data/16-dec/parsed_retweets.json processed (0.39369988441467285 sec)
(4%): 900000 lines in ../data/16-dec/parsed_retweets.json processed (0.3447408676147461 sec)
(4%): 1000000 lines in ../data/16-dec/parsed_retweets.json processed (0.3477671146392822 sec)
(4%): 1100000 lines in ../data/16-

In [8]:
retweet_df.timestamp.min()

'2020-10-23T16:59:58Z'

In [6]:
old_tweet_df, recent_tweet_df = create_tweet_df(
    # retweet_df.timestamp.min()
    '2020-10-23T16:59:58Z', 
    CRAWLED_TERMS, 
    data_dir=DATA_DIR
)

Loading 2696807 json lines
(4%): 100000 lines in ../data/14-nov/parsed_tweets.json processed (2.0737709999084473 sec)
(7%): 200000 lines in ../data/14-nov/parsed_tweets.json processed (2.269927978515625 sec)
(11%): 300000 lines in ../data/14-nov/parsed_tweets.json processed (2.6733357906341553 sec)
(15%): 400000 lines in ../data/14-nov/parsed_tweets.json processed (2.2250869274139404 sec)
(19%): 500000 lines in ../data/14-nov/parsed_tweets.json processed (2.583611011505127 sec)
(22%): 600000 lines in ../data/14-nov/parsed_tweets.json processed (2.8332784175872803 sec)
(26%): 700000 lines in ../data/14-nov/parsed_tweets.json processed (1.5548789501190186 sec)
(30%): 800000 lines in ../data/14-nov/parsed_tweets.json processed (3.0940160751342773 sec)
(33%): 900000 lines in ../data/14-nov/parsed_tweets.json processed (1.6221098899841309 sec)
(37%): 1000000 lines in ../data/14-nov/parsed_tweets.json processed (3.6446330547332764 sec)
(41%): 1100000 lines in ../data/14-nov/parsed_tweets.jso

In [8]:
df_users = create_user_df(data_dir=DATA_DIR)
df_users.info()

Loading 1388621 json lines
(7%): 100000 lines in ../data/16-dec/parsed_users.json processed (6.144344091415405 sec)
(14%): 200000 lines in ../data/16-dec/parsed_users.json processed (4.8742687702178955 sec)
(22%): 300000 lines in ../data/16-dec/parsed_users.json processed (4.657926797866821 sec)
(29%): 400000 lines in ../data/16-dec/parsed_users.json processed (4.414881229400635 sec)
(36%): 500000 lines in ../data/16-dec/parsed_users.json processed (4.135060787200928 sec)
(43%): 600000 lines in ../data/16-dec/parsed_users.json processed (4.198259353637695 sec)
(50%): 700000 lines in ../data/16-dec/parsed_users.json processed (4.581743001937866 sec)
(58%): 800000 lines in ../data/16-dec/parsed_users.json processed (4.835165977478027 sec)
(65%): 900000 lines in ../data/16-dec/parsed_users.json processed (5.120298862457275 sec)
(72%): 1000000 lines in ../data/16-dec/parsed_users.json processed (4.460118055343628 sec)
(79%): 1100000 lines in ../data/16-dec/parsed_users.json processed (4.70

In [9]:
from data_tools import lookup_parsed_data, load_parsed_data
def create_media_df(tweet_df, data_dir="../data/14-nov/"):
    tweet_df_with_media = tweet_df[tweet_df['hasMedia'] == True].set_index('datastore_id')
    # Preserve types when joining
    col_types = tweet_df_with_media.select_dtypes(include=['int', 'int32']).dtypes
    media_df = load_parsed_data(
        data_dir + "/parsed_media.json",
    ).drop_duplicates("media_id").set_index("tweet_id")
    df_media_with_tweets = media_df.join(tweet_df_with_media, on='tweet_id')
    for col, col_type in col_types.iteritems():
        df_media_with_tweets[col] = df_media_with_tweets[col].fillna(0).astype(col_type)

    return df_media_with_tweets

df_media_with_tweets = create_media_df(recent_tweet_df, data_dir=DATA_DIR)
df_media_with_tweets.info()

Loading 389799 json lines
(26%): 100000 lines in ../data/16-dec//parsed_media.json processed (0.3682749271392822 sec)
(51%): 200000 lines in ../data/16-dec//parsed_media.json processed (0.31970787048339844 sec)
(77%): 300000 lines in ../data/16-dec//parsed_media.json processed (0.3132140636444092 sec)
Done loading ../data/16-dec//parsed_media.json
389799 lines in ../data/16-dec//parsed_media.json processed (1.2917020320892334 sec)
<class 'pandas.core.frame.DataFrame'>
Index: 196517 entries, 1339030097194483712 to 1332824783738839040
Data columns (total 49 columns):
 #   Column                      Non-Null Count   Dtype           
---  ------                      --------------   -----           
 0   datastore_id                196517 non-null  object          
 1   type                        196517 non-null  object          
 2   media_id                    196517 non-null  object          
 3   media_url                   196517 non-null  object          
 4   urls                 

In [9]:
def create_crawled_terms_df(crawled_terms, tweet_df):
    crawled_terms_stats = []

    for term in crawled_terms:
        if term in tweet_df.columns:
            stats = {}
            stats["term"] = term
            stats["tweet count"] = tweet_df[term].value_counts().values[1]
            crawled_terms_stats.append(stats)

    crawled_terms_df = pd.DataFrame(crawled_terms_stats).sort_values(
        by=["tweet count"], ascending=False
    )

    return crawled_terms_df

crawled_terms_df = create_crawled_terms_df(CRAWLED_TERMS, recent_tweet_df)
crawled_terms_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 35 to 15
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   term         36 non-null     object
 1   tweet count  36 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 864.0+ bytes


## Basic stats & Coverage

In [10]:

coverage_stats = {}

coverage_stats["old_tweet_count"] = len(old_tweet_df.index)
coverage_stats["recent_tweet_count"] = len(recent_tweet_df.index)
coverage_stats["total_tweet_count"] = coverage_stats["recent_tweet_count"] + coverage_stats["old_tweet_count"]
coverage_stats["retweet_count"] = len(retweet_df.index)
coverage_stats["user_count"] = len(df_users.index)

coverage_stats["earliest_tweet"] = recent_tweet_df.timestamp.min()
coverage_stats["latest_tweet"] = recent_tweet_df.timestamp.max()
coverage_stats["earliest_retweet"] = retweet_df.timestamp.min()
coverage_stats["latest_retweet"] = retweet_df.timestamp.max()

coverage_stats

{'old_tweet_count': 5902,
 'recent_tweet_count': 7603103,
 'total_tweet_count': 7609005,
 'retweet_count': 25566698,
 'user_count': 1388621,
 'earliest_tweet': '2008-11-05T02:44:00Z',
 'latest_tweet': '2020-12-16T13:08:49Z',
 'earliest_retweet': '2020-10-23T16:59:58Z',
 'latest_retweet': '2020-12-16T13:42:14Z'}

## Terms grouped by hour

In [11]:
df_aggregated_by_hour = aggregate_counts_by_hour(recent_tweet_df, retweet_df, crawled_terms_df["term"].values)
df_aggregated_by_hour.head()

,tweet count,retweet count,voter fraud,election fraud,#stopthesteal,#voterfraud,#electionfraud,election interference,ballot harvesting,ballot fraud,...,hacked voting machine,pre-filled ballot,#ilhanomarballotharvesting,#ballotvoterfraud,#votebymailfraud,#nomailinvoting,#ilhanomarvoterfraud,#hackedvotingmachines,#discardedballots,#stopgopvoterfraud
date,,,,,,,,,,,,,,,,,,,,,
2020-10-23 17:00:00+00:00,306,681.0,179,4.0,0.0,24.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-10-23 18:00:00+00:00,419,1272.0,238,9.0,0.0,22.0,1.0,0.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-10-23 19:00:00+00:00,409,561.0,250,8.0,1.0,50.0,0.0,2.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-10-23 20:00:00+00:00,645,932.0,372,18.0,0.0,77.0,2.0,2.0,6.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-10-23 21:00:00+00:00,539,847.0,322,11.0,0.0,57.0,1.0,0.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Most common hashtags

In [12]:
df_most_common_hashtags = aggregate_most_common_hashtags(recent_tweet_df, crawled_terms_df["term"].values)
df_most_common_hashtags.head()

,all tweets,voter fraud,election fraud,#stopthesteal,#voterfraud,#electionfraud,election interference,ballot harvesting,ballot fraud,#electioninterference,...,hacked voting machine,pre-filled ballot,#ilhanomarballotharvesting,#ballotvoterfraud,#votebymailfraud,#nomailinvoting,#ilhanomarvoterfraud,#hackedvotingmachines,#discardedballots,#stopgopvoterfraud
hashtag,,,,,,,,,,,,,,,,,,,,,
#1,0,741,530,0,0,0,47,21,18,0,...,0,0,0,0,0,0,0,0,0,0
#12news,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#1a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#2,0,0,0,0,0,0,0,12,0,0,...,0,0,0,0,0,0,0,0,0,0
#2020election,9507,934,701,2282,2680,2608,62,14,19,340,...,0,0,0,2,0,0,0,0,0,0


## Most common tokens

In [17]:
from collections import Counter
from spacy.lang.en.stop_words import STOP_WORDS
STOP_WORDS = STOP_WORDS.union({"pron", "", " ", "”", "“", "🇺"})

def include_token(token):
    return token not in STOP_WORDS and not token.startswith("hashtag")

def create_most_common_tokens_df(df_tweets, count_label, k=100):
    counted_tokens = Counter(
        [
            token
            for tokens in df_tweets["tokens"]
            for token in tokens
            if include_token(token)
        ]
    )
    return pd.DataFrame(
        counted_tokens.most_common(k), columns=["token", count_label]
    ).set_index("token")

def aggregate_most_common_tokens(df_tweets, crawled_terms, k=100):
    df_most_common_tokens = create_most_common_tokens_df(df_tweets, count_label="all tweets", k=k)
    for term in crawled_terms:
        filtered_by_crawled_term = df_tweets[
            df_tweets[term] == 1
        ]
        df_most_common_tokens = df_most_common_tokens.join(
            create_most_common_tokens_df(filtered_by_crawled_term, count_label=term, k=k),
            how='outer'
        )
    return df_most_common_tokens.fillna(0).astype(int)


df_most_common_tokens = aggregate_most_common_tokens(recent_tweet_df, crawled_terms_df["term"].values)
df_most_common_tokens.head()

,all tweets,voter fraud,election fraud,#stopthesteal,#voterfraud,#electionfraud,election interference,ballot harvesting,ballot fraud,#electioninterference,...,hacked voting machine,pre-filled ballot,#ilhanomarballotharvesting,#ballotvoterfraud,#votebymailfraud,#nomailinvoting,#ilhanomarvoterfraud,#hackedvotingmachines,#discardedballots,#stopgopvoterfraud
token,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,17,8,0,0,1,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,206,...,0,0,0,0,0,0,0,0,0,0
100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1000s,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


## Co-occurrence

In [ ]:
crawled_term_threshold = 5000
filtered_crawled_terms = crawled_terms_df[
    crawled_terms_df["tweet count"] > crawled_term_threshold
]
terms_in_df = [term for term in filtered_crawled_terms["term"]]
crawled_terms_tweet_df = (
    recent_tweet_df[terms_in_df].sparse.to_dense().astype("int32")
)
df_cooccurrence = crawled_terms_tweet_df.T.dot(crawled_terms_tweet_df)

## Quote Tweets

In [11]:
df_quote_tweets = recent_tweet_df[recent_tweet_df["quote_tweet"].notna()]

In [28]:
# Add quote tweet count to coverage stats and update earliest_tweet
with open(EXPORT_DIR + "coverage_stats.pickle", "rb") as f:
    coverage_stats = pickle.load(f)
    coverage_stats["quote_tweet_count"] = df_quote_tweets.shape[0]
    coverage_stats["earliest_tweet"] = recent_tweet_df["timestamp"].min()


In [29]:
print(coverage_stats)

{'old_tweet_count': 5902, 'recent_tweet_count': 7603103, 'total_tweet_count': 7609005, 'retweet_count': 25566698, 'user_count': 1388621, 'earliest_tweet': '2020-10-23T17:00:04Z', 'latest_tweet': '2020-12-16T13:08:49Z', 'earliest_retweet': '2020-10-23T16:59:58Z', 'latest_retweet': '2020-12-16T13:42:14Z', 'quote_tweet_count': 3821579}


## Top tweets by week


In [16]:
import datetime as dt
tweets = recent_tweet_df
tweets['WeekDate'] = pd.to_datetime(tweets.timestamp).dt.to_period('W').apply(lambda r: r.start_time)
weeks = tweets['WeekDate'].unique()

def extract_weekly_tweets(df_week, N=25):
    media_tweets = df_week[df_week["hasMedia"]]
    top_retweeted = df_week.nlargest(N, "retweet_count")
    top_quoted = df_week.nlargest(N, "quote_count")
    media_top_retweeted = media_tweets.nlargest(N, "retweet_count")
    media_top_quoted = media_tweets.nlargest(N, "quote_count")
    return (top_retweeted.merge(top_quoted, how="outer")
                        .merge(media_top_retweeted, how="outer")
                        .merge(media_top_quoted, how="outer"))

columns = ["datastore_id", "quote_count", "retweet_count", "hasMedia", "text", "user", "timestamp"]
weekly_top_tweets = (tweets
    .groupby("WeekDate")[columns]
    .apply(lambda df: extract_weekly_tweets(df))
#  Merge with user df and sort by week date, retweet count
    .merge(df_users[["handle", "name"]], how='inner', left_on="user", right_index=True)
    .set_index("user", append=True)
    .sort_values(by=['WeekDate', 'retweet_count'], ascending=[True, False]))
weekly_top_tweets.head()

datastore_id  quote_count  retweet_count  \
WeekDate     user                                                         
2020-10-19 0 259001548  1320117044612943872        10199          34530   
           1 341194704  1320107370312323073         9158          22217   
           2 25073877   1320168905856397326          941          17118   
           3 137752344  1320161049769889792         2846          15897   
           4 39349894   1320160474672095232         3736          14948   

                       hasMedia  \
WeekDate     user                 
2020-10-19 0 259001548    False   
           1 341194704    False   
           2 25073877     False   
           3 137752344    False   
           4 39349894     False   

                                                                     text  \
WeekDate     user                                                           
2020-10-19 0 259001548  🚨🚨 BIDEN ADMITS TO VOTER FRAUD! 🚨🚨\n\n@JoeBide...   
           1 341194704  👀👀 \n\nJoe Biden brags about having “the most ...   
           2 25073877   Law Enforcement is watching and involved. So d...   
           3 137752344  I hope you’ve been hacked, @GovMikeHuckabee. T...   
           4 39349894   😳😳😳\n\nBiden: “We have put together the most e...   

                                   timestamp           handle  \
WeekDate     user                                               
2020-10-19 0 259001548  2020-10-24T21:36:44Z  kayleighmcenany   
           1 341194704  2020-10-24T20:58:18Z       SteveGuest   
           2 25073877   2020-10-25T01:02:49Z  realDonaldTrump   
           3 137752344  2020-10-25T00:31:36Z  EllenLWeintraub   
           4 39349894   2020-10-25T00:29:19Z        EricTrump   

                                       name  
WeekDate     user                            
2020-10-19 0 259001548     Kayleigh McEnany  
           1 341194704          Steve Guest  
           2 25073877       Donald J. Trump  
           3 137752344  Ellen L 😷 Weintraub  
           4 39349894            Eric Trump

In [17]:
weekly_top_tweets.shape

(554, 8)

In [103]:
print(len(weeks))

9


## Top users by week


In [19]:
import datetime as dt
tweets = recent_tweet_df
tweets['WeekDate'] = pd.to_datetime(tweets.timestamp).dt.to_period('W').apply(lambda r: r.start_time)
weeks = tweets['WeekDate'].unique()

def extract_top_users(df_week, N=25):
    top_retweeted = df_week.nlargest(N, "retweeted_count").reset_index(level=["user"])
    top_quoted = df_week.nlargest(N, "quoted_count").reset_index(level=["user"])
    most_tweets = df_week.nlargest(N, "all_tweet_count").reset_index(level=["user"])
    most_media_tweets = df_week.nlargest(N, "media_tweet_count").reset_index(level=["user"])
    most_quote_tweets = df_week.nlargest(N, "quote_tweet_count").reset_index(level=["user"])
    return (top_retweeted.merge(top_quoted, how="outer")
                         .merge(most_tweets, how="outer")
                         .merge(most_media_tweets, how="outer")
                         .merge(most_quote_tweets, how="outer")).set_index("user")

columns = tweets.columns
weekly_top_users = (tweets.groupby(["WeekDate", "user"])
    .agg(
        retweeted_count=('retweet_count', 'sum'), 
        quoted_count=('quote_count', 'sum'), 
        all_tweet_count=('datastore_id', 'count'),
        media_tweet_count=('hasMedia', 'sum'),
        quote_tweet_count=('quote_tweet', 'count')
    )
    .groupby(level=0, axis=0)
    .apply(lambda df: extract_top_users(df))
    #  Merge with user df and sort by week date, retweet count
    .reset_index(level=["user"])
    .merge(df_users, how='inner', left_on="user", right_index=True)
    .set_index("user", append=True)
    .sort_values(by=['WeekDate', 'media_tweet_count'], ascending=[True, False]))

weekly_top_users.head()

retweeted_count  quoted_count  \
WeekDate   user                                                 
2020-10-19 1316429019198500865               11             3   
           1225026641614774273                0             0   
           1225835542765395968                0             0   
           2937103496                         0             0   
           410022823                          0             0   

                                all_tweet_count  media_tweet_count  \
WeekDate   user                                                      
2020-10-19 1316429019198500865               33                 29   
           1225026641614774273               12                 11   
           1225835542765395968               10                 10   
           2937103496                         9                  8   
           410022823                         13                  8   

                                quote_tweet_count  protected friends  \
WeekDate   user                                                        
2020-10-19 1316429019198500865                  1      False    None   
           1225026641614774273                  0      False    None   
           1225835542765395968                  0      False    None   
           2937103496                           0      False    None   
           410022823                           13      False    None   

                                          created_at                     name  \
WeekDate   user                                                                 
2020-10-19 1316429019198500865  2020-10-14T17:22:27Z          Chalmatian70043   
           1225026641614774273  2020-02-05T12:01:42Z        Philip Castilleja   
           1225835542765395968  2020-02-07T17:35:59Z            FedupAmerican   
           2937103496           2014-12-20T13:06:56Z  Gee.Frank is back y'all   
           410022823            2011-11-11T14:49:06Z                    Alan7   

                                friends_count  verified  followers_count  \
WeekDate   user                                                            
2020-10-19 1316429019198500865             56     False                8   
           1225026641614774273             68     False               31   
           1225835542765395968           4971     False             4477   
           2937103496                    1927     False             2592   
           410022823                     4982     False             3349   

                                   location  followed_cnts           handle  \
WeekDate   user                                                               
2020-10-19 1316429019198500865         None              0  chalmatian70043   
           1225026641614774273         None              0  PhilipCastille2   
           1225835542765395968         None              0  FedupAm48219051   
           2937103496           Alaska, USA              0    Frank_N_Meems   
           410022823                   None              0         gohavfun   

                                 url  
WeekDate   user                       
2020-10-19 1316429019198500865  None  
           1225026641614774273  None  
           1225835542765395968  None  
           2937103496           None  
           410022823            None

In [20]:
weekly_top_users.shape

(804, 16)

## Export

In [ ]:
import pickle

df_users.to_pickle(EXPORT_DIR + "df_users.pickle")
retweet_df.to_pickle(EXPORT_DIR + "df_retweets.pickle")
recent_tweet_df.to_pickle(EXPORT_DIR + "df_recent_tweets.pickle")
old_tweet_df.to_pickle(EXPORT_DIR + "df_old_tweets.pickle")
df_aggregated_by_hour.to_pickle(EXPORT_DIR + "df_counts_by_hour.pickle")
crawled_terms_df.to_pickle(EXPORT_DIR + "df_crawled_terms.pickle")
df_most_common_hashtags.to_pickle(EXPORT_DIR + "df_most_common_hashtags.pickle")
df_most_common_tokens.to_pickle(EXPORT_DIR + "df_most_common_tokens.pickle")
df_cooccurrence.to_pickle(EXPORT_DIR + "df_cooccurrence.pickle")
df_media_with_tweets.to_pickle(EXPORT_DIR + "df_media_with_tweets.pickle")
df_quote_tweets.to_pickle(EXPORT_DIR + "df_quote_tweets.pickle")
weekly_top_tweets.to_pickle(EXPORT_DIR + "df_weekly_top_tweets.pickle")
weekly_top_users.to_pickle(EXPORT_DIR + "df_weekly_top_users.pickle")

with open(EXPORT_DIR + "coverage_stats.pickle", "wb") as f:
    pickle.dump(coverage_stats, f)

In [15]:
df_quote_tweets.to_pickle(EXPORT_DIR + "df_quote_tweets.pickle")

In [30]:
with open(EXPORT_DIR + "coverage_stats.pickle", "wb") as f:
    pickle.dump(coverage_stats, f)

In [116]:
weekly_top_tweets.to_pickle(EXPORT_DIR + "df_weekly_top_tweets.pickle")
weekly_top_users.to_pickle(EXPORT_DIR + "df_weekly_top_users.pickle")

In [8]:
recent_tweet_df.to_pickle(EXPORT_DIR + "df_recent_tweets.pickle")
old_tweet_df.to_pickle(EXPORT_DIR + "df_old_tweets.pickle")

In [21]:
weekly_top_tweets.to_pickle(EXPORT_DIR + "df_weekly_top_tweets.pickle")